In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#-*- coding: utf-8 -*-

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

### 1. Data Set management 

In [ ]:
from PIL import Image
root_dir = "/content/drive/My Drive/sensorimage/Full_image/"
image_dir = root_dir + 'boar'
cropimage_dir = "/content/drive/My Drive/sensorimage/Clip_image/"
files = glob.glob(image_dir + "/" + "*.JPG")
  # "Full_image 에 있는 이미지를 잘라서 Clip_image 에 넣어라" 는 명령

for i, f in enumerate(files):
  # 이미지 로딩 (3)
  img = Image.open(f)
  width , height = img.size
  # 이미지 자르기
  cropImage = img.crop((0, 0, width, height-200))
  nameonly=os.path.splitext(f)
  new_name=os.path.split(nameonly[0])
  print(str(i) + '_' +new_name[1])
  cropImage.save(cropimage_dir+'boar/'+new_name[1]+'.JPG')


In [ ]:
from PIL import Image
root_dir = "/content/drive/My Drive/sensorimage/Full_image/"
image_dir = root_dir + 'others'
cropimage_dir = "/content/drive/My Drive/sensorimage/Clip_image/"
files = glob.glob(image_dir + "/" + "*.JPG")

for i, f in enumerate(files):
  # 이미지 로딩 (3)
  img = Image.open(f)
  width , height = img.size
  # 이미지 자르기
  cropImage = img.crop((0, 0, width, height-200))
  nameonly=os.path.splitext(f)
  new_name=os.path.split(nameonly[0])
  print(str(i) + '_' +new_name[1])
  cropImage.save(cropimage_dir+'others/'+new_name[1]+'.JPG')

In [ ]:
#others샘플링
import random
import shutil

# (1) 숫자리스트 샘플링
numlist=list(range(1,2287))
s = random.sample(numlist, 228)
#print(s)

for i in s:
  try:
    fn= '{0:04d}'.format(i)
    print(cropimage_dir+'others/N' + fn + '.JPG')
    shutil.move(cropimage_dir+'others/N' + fn + '.JPG', cropimage_dir+'testdata/')
  except FileNotFoundError:
    pass    

In [ ]:
#boar샘플링
import random
import shutil

# (1) 숫자리스트 샘플링
numlist=list(range(1,2287))
s = random.sample(numlist, 228)
#print(s)

for i in s:
  try:
    fn= '{0:04d}'.format(i)
    print(cropimage_dir+'boar/Y' + fn + '.JPG')
    shutil.move(cropimage_dir+'boar/Y' + fn + '.JPG', cropimage_dir+'testdata/')
  except FileNotFoundError:
    pass    

In [ ]:
root_dir = "/content/drive/My Drive/sensorimage/Clip_image/"
# 카테고리 명령 지정 (1)
categories = ["boar", "others"] # deer, 등등 추가가능
nb_classes = len(categories)

In [ ]:
# 이미지 크기 지정 (2)
image_width = 500
image_height = 300

In [ ]:
## 데이터 변수
X = [] # 이미지 데이터, 이미지를 조각내서 넣음
Y = [] # 레이블 데이터, 그래서 조각낸 이미지가 멧돼지인지 아닌지 판별

In [ ]:
for idx, category in enumerate(categories):
    image_dir = root_dir + category
    files = glob.glob(image_dir + "/" + "*.JPG")
    print(image_dir + "/" +"*.JPG")

    for i, f in enumerate(files):
        # 이미지 로딩 (3)
        print(image_dir + "_"+str(i))
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_width, image_height))
        data = np.asarray(img)
        X.append(data)
        Y.append(idx)

X = np.array(X)
Y = np.array(Y)

In [ ]:
# 학습 데이터와 테스트 데이터 나누기 (4)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

xy = (X_train, X_test, Y_train, Y_test)
# 데이터 파일 저장 (5)
#from pyocser import ocdumps, ocloads
#pickle.dump(d, open("file", 'w'), protocol=4)
np.save(root_dir + "boar_others_500_300.npy", xy, allow_pickle=True)
#여기까지

/usr/local/lib/python3.8/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [ ]:
#file 복사
#cp root_dir+"/boar_N/*.jpg" root_dir+"/boar_all/"

import shutil
shutil.copytree(root_dir+"boar_N/", root_dir+"boar_all/")

'/content/drive/My Drive/sensorimage/Clip_image/boar_all/'

In [ ]:
from distutils.dir_util import copy_tree
copy_tree(root_dir+"boar/", root_dir+"boar_all/")

## classification ALEX net

In [ ]:
## 사용할 모델 라이브러리 import
import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
root_dir = "/content/drive/My Drive/"
# 카테고리 명령 지정 (1)
categories = ["boar", "others"]
nb_classes = len(categories)
image_size = 128

In [ ]:
def load_dataset():
    x_train, x_test, y_train, y_test = np.load(root_dir+"boar_others_500_300.npy", allow_pickle=True)
    #x_train, x_test, y_train, y_test = train_test_split(X, Y)
    x_train = x_train.astype("float") / 256
    x_test = x_test.astype("float") / 256
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return  x_train, x_test, y_train, y_test

In [ ]:
# 모델 구성  (2)
def build_model(in_shape):
    model = Sequential()
    model.add(Convolution2D(64, 3, 3, padding='same', 
                input_shape=in_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Convolution2D(128, 3, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(128, 3, 3))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss='binary_crossentropy', 
                    optimizer='rmsprop', 
                    metrics=['accuracy'])

    return model

In [ ]:
import datetime


# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(log_dir)
# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
# 모델 학습을 수행하고 저장된 모델을 파일로 저장  (3)
def model_train(x, y):
    model = build_model(x.shape[1:])
    model.fit(x, y, batch_size=32, epochs=30,
              callbacks=[tensorboard_callback])

    return model

In [ ]:
# 모델 평가하기  (4)
def model_eval(model, x, y):
    score = model.evaluate(x, y)
    print('loss=', score[0])
    print('accuracy=', score[1])

In [ ]:
x_train, x_test, y_train, y_test = load_dataset()

In [ ]:
model = model_train(x_train, y_train)

In [ ]:
model_eval(model, x_test, y_test)

In [ ]:
# 모델 학습 및 평가
x_train, x_test, y_train, y_test = load_dataset()
model = model_train(x_train, y_train)
model_eval(model, x_test, y_test)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir {log_dir}

In [ ]:
# 모델 저장
model.save(root_dir+"boar_model_500_300.h5")   # saving model 

## 3. detect and test

In [ ]:
root_dir = "/content/drive/My Drive/sensorimage/Clip_image/"

In [ ]:
#-*- coding: utf-8 -*-

import sys, os
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
from keras.models import load_model
from PIL import Image
import os, glob
import numpy as np

In [ ]:
# 테스트 이미지 목록 (1)

image_files = glob.glob(root_dir+"testdata/" +  "*.JPG")
image_size1 = 300
image_size2 = 500
nb_classes = len(image_files)
categories = ["boar", "others"]

In [ ]:
X = []
files = []
# 이미지 불러오기 (2)
for fname in image_files:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((image_size2, image_size1))
    in_data = np.asarray(img)
    in_data = in_data.astype("float") / 256
    X.append(in_data)
    files.append(fname)

X = np.array(X)

In [ ]:
# 모델 파일 읽어오기  (3)
model = load_model(root_dir+'boar_model_500_300.h5') # load model

In [ ]:
# 예측 실행  (4)
pre = model.predict(X)   # model predict

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=range(3))
df.columns=['test_file','예측값','예측확률']

In [ ]:
# saving predicted file 
for i, p in enumerate(pre):
    y = p.argmax()
    #print("입력:", files[i])
    #print("예측:", "[", y, "]", categories[y], "/ Score",p[y])
    df.loc[i]=[files[i],categories[y],p[y]]

df    

df.to_excel(root_dir+"boar_"+str(image_size1)+"_"+str(image_size2)+".xlsx")

########### The End ###############